In [23]:
import os
import numpy as np
import pandas as pd

data_path = '../../data/annotations/group_mention_categorization/social-group-mention-categorization-coder-training'

In [66]:
fp = os.path.join(data_path, 'parsed', 'annotations.tsv')
annotations = pd.read_csv(fp, sep='\t')
annotators = annotations.annotator.unique()

annotations.loc[annotations.q_category.isna(), 'q_category'] = -1.0
annotations.loc[annotations.q_id == 'universal_attributes', 'q_category'] = -1.0

In [67]:
fp = os.path.join(data_path, 'parsed', 'consolidated_annotations.tsv')
consolidated = pd.read_csv(fp, sep='\t')

In [69]:
annotations.loc[annotations.q_id == 'universal_attributes', ['category', 'q_category']].value_counts(dropna=False)

category  q_category
NaN       -1.0          600
Name: count, dtype: int64

In [70]:
consolidated.loc[consolidated.q_id == 'universal_attributes', ['category', 'q_category']].value_counts(dropna=False)

category  q_category
NaN       -1.0          300
Name: count, dtype: int64

In [105]:
text_cols = ['mention_id', 'text', 'mention']
cols = ['mention_id', 'q_id', 'q_category', 'category']

In [113]:
tmp = consolidated.copy(deep=True)
tmp.loc[tmp.category == 'other', 'q_category'] = 99.0

# concate values in category where label=='Yes' into string of comma separated values, sorting by q_category
def concat(x):
    if x.category.isna().all():
        return x.label.iloc[0]
    x = x[x.label == 'Yes']
    if len(x) == 0:
        return '-/-'
    x.sort_values('q_category', inplace=True)
    return ', '.join(x.category)
    
consolidated_wider = tmp.groupby(['mention_id', 'q_id']).apply(concat).reset_index().rename(columns={0: 'consolidated_labels'})

/var/folders/_c/cm0nk6y92rz2l6ct3npgw9tr0000gn/T/ipykernel_18174/2075745738.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  consolidated_wider = tmp.groupby(['mention_id', 'q_id']).apply(concat).reset_index().rename(columns={0: 'consolidated_labels'})


In [120]:
for annotator, annons in annotations.groupby('annotator'):
    print(annotator)
    tmp = consolidated[cols + ['label']].merge(
        annons[cols + ['response']],
        on=cols,
        how='left'
    )

    tmp.loc[np.logical_and(tmp.q_id == 'universal_attributes', tmp.response.isna()), 'response'] = 'No'

    idxs = np.logical_and(tmp.q_id == 'non-economic_attributes', tmp.category == 'shared values/mentalities')
    idxs = np.logical_and(idxs, tmp.label == 'No')
    tmp = tmp[~idxs]

    # subset to only disagreements
    disagreements = tmp[tmp.label != tmp.response]

    # join wider labels (for annotator to see how we decided eventually)
    disagreements = disagreements.merge(consolidated_wider, on=['mention_id', 'q_id'], how='left')

    # join text and mention
    disagreements = disagreements.merge(consolidated[text_cols].drop_duplicates(), on='mention_id', how='left')

    disagreements.sort_values(['mention_id', 'q_id', 'q_category'], inplace=True)
    disagreements = disagreements[text_cols + ['q_id', 'category', 'label', 'response', 'consolidated_labels']]
    disagreements.rename(columns={'q_id': 'dimension'}, inplace=True)

    fp = os.path.join(data_path, 'parsed', f'feedback_{annotator.lower()}.tsv')
    disagreements.to_csv(fp, sep='\t', index=False)

Eichholz
Ford
